# Content Based Filtering

## Problem Statement:
In the Summer of 2022, we introduced a matching algorithm based off of Gale Shapley to match EFL students with startups. For the algorithm to work as intended both students and startups had to rank each other. While students are expected to match as many startups as possible that they want to work with, having startups rank students was deemed as added friction especially since they aren't guaranteed their best match. As a result, we want to have startups list out some sort of preferences for what they are looking from students instead of ranking students.

## Solution:
One way to achieve the following is to implement a content based filtering algorithm used in early recommendation systems. Recommendation systems have been integrated well into our lives since the last decade- we have product recommendations come in based on products we have previously bought or looked at, and movie recommendations based off the movies we previously watched. While these algorithms are more advanced, a simple content based filtering algorithm does the following:

Given a set of profiles and a feature set, we conduct a similarity metric for every profile against the feature set, and report the highest match.

Refer: https://developers.google.com/machine-learning/recommendation/content-based/basics#:~:text=Content%2Dbased%20filtering%20uses%20item,previous%20actions%20or%20explicit%20feedback.

To understand dot products, you can refer to [this](https://en.wikipedia.org/wiki/Dot_product) to learn more.

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm

# Let's review a Matrix Operation

Let us assume we have a 3x2 matrix A, where \
A=$\big(\begin{bmatrix}
  1 & 2\\
  3 & 4 \\
  5 & 6
\end{bmatrix}\big)$, and a 2x1 matrix B, where B=$\big(\begin{bmatrix}
  1 \\
  3
\end{bmatrix}\big)$.

Here we can consider every row in matrix A to be a row for each student and every row in matrix B to a feature for one startup.

A dot product between one student and the startup would look like the following:
$student_1=\big(\begin{bmatrix}
  1 & 2
\end{bmatrix}\big)$ and $startup =\big(\begin{bmatrix}
  1 \\
  3
\end{bmatrix}\big)$

$= \big(\begin{bmatrix}
  1 & 2
\end{bmatrix}\big)$ . $\big(\begin{bmatrix}
  1 \\
  3
\end{bmatrix}\big) = \big(\begin{bmatrix}
  1*1 + & 2*3
\end{bmatrix}\big) = 7$



Now if we do this for every student, we will perform the following operation where we will every row in matrix A and do a dot product with B, which is also similar to a matrix multiplication:

A dot B = $\big(\begin{bmatrix}
  1 & 2\\
  3 & 4 \\
  5 & 6
\end{bmatrix}\big) . \big(\begin{bmatrix}
  1\\
  3
\end{bmatrix}\big) = \big(\begin{bmatrix}
  7\\
  15\\
  23
\end{bmatrix}\big)$

What's important to note here is that the resulting matrix $\big(\begin{bmatrix}
  7\\
  15\\
  23
\end{bmatrix}\big)$ corresponds to the dot product result for every row in matrix A.

Below, we will demo this operation in code.

# So what's a Dot Product?

Dot Product is a term taken out of Linear Algebra, which in simple words measures the similarity between two vectors. For our use case, it is a measure for closeness of what a startup prefers against a student's profile. To demonstrate, we can take the above example and turn it into an example for our usecase. Instead of having numbers 1,2,3.. we will convert them to 1's and 0's. Here 1 represents a boolean value which says something exists, and 0 otherwise.



Let us assume we have a 3x2 matrix Student_Profile, where \
Student_Profile=$\big(\begin{bmatrix}
  1 & 0\\
  1 & 1 \\
  0 & 0
\end{bmatrix}\big)$, and a 2x1 matrix startup_interest, where startup_interest=$\big(\begin{bmatrix}
  1 \\
  1
\end{bmatrix}\big)$.

For now, lets assume that the startup prefers a deep tech student and a MBA student which is denoted by $\big(\begin{bmatrix}
  1=deep tech \\
  1=MBA
\end{bmatrix}\big)$. For every row in Student_Profile, the first column corresponds to whether the student is interest or has a background in deep tech or not, represented by a 1 or 0, and the second column represents whether they have a background in MBA or not.

Here we can consider every row in matrix A to be a row for each student and every row in matrix B to a feature for one startup.

A dot product between one student and the startup would look like the following:
$student_1=\big(\begin{bmatrix}
  1 & 0
\end{bmatrix}\big)$ and $startup =\big(\begin{bmatrix}
  1 \\
  1
\end{bmatrix}\big)$

$= \big(\begin{bmatrix}
  1 & 0
\end{bmatrix}\big)$ . $\big(\begin{bmatrix}
  1 \\
  1
\end{bmatrix}\big) = \big(\begin{bmatrix}
  1*1 + & 0*1
\end{bmatrix}\big) = 1$



Now if we do this for every student, we will perform the following operation where we will every row in matrix Student_profile and do a dot product with startup, which is also similar to a matrix multiplication:

Student_profile dot Startup = $\big(\begin{bmatrix}
  1 & 0\\
  1 & 1 \\
  0 & 0
\end{bmatrix}\big) . \big(\begin{bmatrix}
  1\\
  1
\end{bmatrix}\big) = \big(\begin{bmatrix}
  1\\
  2\\
  0
\end{bmatrix}\big)$

What the resulting vector indicates to us is that, both student 1 and 2 are preferred by the startup but student 2 is more preferred because student 2 ticks both the boxes for what the startup prefers. That's why we get a higher similarity. And as usuall, the student who doesnt have deep tech or MBA is not preferred by the startup, and as a result has a 0 dot product.

#Binary Values

Lets say, the array goes like:

[deep tech, digital tech, life science, MBA, startup work experience, python, marketing, strategy, Law]

In [2]:
startup = np.array([1,0,0,1,1,1,1,1,1])
startup

array([1, 0, 0, 1, 1, 1, 1, 1, 1])

In [3]:
# a student profile array is created, where every row is a student and every col corresponds to the col in startup array indicating whether
# there is a match between the startup and student, indicated by 1, or not, indicated by 0.
studentProfile = [
    [1,0,0,0,1,1,1,0,0],
    [1,0,0,1,1,1,1,1,0],
    [1,1,0,1,0,1,0,1,1],
    [0,0,1,1,1,1,0,0,0],
    [1,0,0,0,1,1,0,1,0],
]
studProfile = np.array(studentProfile)
studProfile

array([[1, 0, 0, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 0, 1, 0, 1, 1],
       [0, 0, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 0, 1, 0]])

In [4]:
np.dot(studProfile,startup)

array([4, 6, 5, 3, 4])

To interpret this result, we see that there is high similarity between the 2rd student with this particular startup, followed by the 3rd and so on.

## Cosine Similarity

![link text](https://upload.wikimedia.org/wikipedia/commons/thumb/7/76/Inner-product-angle.svg/440px-Inner-product-angle.svg.png)

To expand upon the dot product metric, we can perform a cosine similarity metric to measure the similarity. The way it works is that, given two vectors like we have been seeing previously, i.e student_profile and startup, we want to find the angle between them represented by θ.

In [5]:
cos_sim = np.dot(studProfile,startup)/(norm(studProfile)*norm(startup))
cos_sim

array([0.3086067 , 0.46291005, 0.38575837, 0.23145502, 0.3086067 ])

To interpret the result above, we say higher values represent higher similarity, and our results should be similar to the dot product result we had before. It suggests that the 2nd student has the highest match followed by the 3rd and so on.

For both the dot product and cosine similarity, we get a resulting array of length 5, which is similar the number of students in student Profile. Which as I said before, the matrix resultant after the dot product corresponds to every row in the student profile

#Continuous Values

Lets say, the array goes like:

[deep tech, digital tech, life science, MBA, startup work experience, python, marketing, strategy, Law]

In [6]:
startup = np.array([1,0,0,0.8,1,0.5,0.83,0.32,0.1])
startup

array([1.  , 0.  , 0.  , 0.8 , 1.  , 0.5 , 0.83, 0.32, 0.1 ])

In [7]:
# a student profile array is created, where every row is a student and every col corresponds to the col in startup array indicating whether
# there is a match between the startup and student, indicated by 1, or not, indicated by 0.
studentProfile = [
    [1,0,0,0,1,1,1,0,0],
    [1,0,0,1,1,1,1,1,0],
    [1,1,0,1,0,1,0,1,1],
    [0,0,1,1,1,1,0,0,0],
    [1,0,0,0,1,1,0,1,0],
]
studProfile = np.array(studentProfile)
studProfile

array([[1, 0, 0, 0, 1, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 1, 1, 1, 0],
       [1, 1, 0, 1, 0, 1, 0, 1, 1],
       [0, 0, 1, 1, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 0, 1, 0]])

In [8]:
np.dot(studProfile,startup)

array([3.33, 4.45, 2.72, 2.3 , 2.82])

In [9]:
startup = np.array([1,0,0,0.8,1,0.5,0.83,0.32,0.1])
startup2 = np.array([1,1,0,0.5,0.8,0.83, 0.5,0.1,0.32])
studentProfile = [
    [1,0,0,0,1,1,1,0,0],
    [1,0,0,1,1,1,1,1,0],
    [1,1,0,1,0,1,0,1,1],
    [0,0,1,1,1,1,0,0,0],
    [1,0,0,0,1,1,0,1,0],
]
studProfile = np.array(studentProfile)
np.dot(studProfile,startup)
cos_sim = np.dot(studProfile,startup)/(norm(studProfile)*norm(startup))
cos_sim2 = np.dot(studProfile,startup2)/(norm(studProfile)*norm(startup2))

In [10]:
print(cos_sim, cos_sim2)

[0.35379276 0.47278612 0.28898388 0.24436137 0.29960828] [0.3218244  0.38351598 0.38557237 0.2190051  0.28069668]


Given that we had the following preferences from the startup: \[1.  , 0.  , 0.  , 0.8 , 1.  , 0.5 , 0.83, 0.32, 0.1] wherein the startup:


1.   Prefers only Deep Tech candidates
2.   80% preference for an MBA Student
3.   High preference for startup experience
4.   50% preference for experience working with python
5.   83% marketing experience
6.   32% strategy experience
7.   10% experience with Law

All the percentages are preference rankings. With that being said, it seems like students 2 and 1 are highly preferened in order.

For student 2, their profile included binary values of: \[1, 0, 0, 1, 1, 1, 1, 1, 0] with background in everything prefered by the startup other than law.
For student 1, they lack strategy and law experience compared to student 3 who has both strategy and law but doesnt have marketing experience which the startup prefers at 83%.

In the Binary example, student 3 was actually second preference when the startup rankings were also binary. This may indicate that we want to ask startups how much weight they put for each of their features



# What do we need?

Currently we are focused on creating a feature list that we can provide our startups. These features are expected to be features that the startups are interested in from students. The more features we add, the more robust our preferences lists will be.

**Questions that we need to ask:**


1.   What features are we interested in providing startups? As we can see, we get richer results when we have some continuity in the features. For example, instead of startups stating yes/no, startups stating how much they want a certain feature yields better results.
      1. One thing we could do is have the startups list what features they think are important, and then ask them how much weight they attach to that.
2.   Given that we all agree on features that we can ask our startups to rank, how do we incorporate these features to build a student profile?
      1. Do we design a form that collects these features from students when they submit their profiles and preferences?
      2. Or, is there a form or data that we can use to build a student profile automatically?





# What's next?

Given that we go forward with this, and we get a similarity score as seen above, we will take the following result:

```python
array([0.35379276, 0.47278612, 0.28898388, 0.24436137, 0.29960828])
       student 1,   student 2,  student 3, student 4,  student 5
```
and sort it to get:
```python
array([0.47278612, 0.35379276, 0.29960828, 0.28898388, 0.24436137])
       student 2,   student 1,  student 5, student 3,  student 4
```
and get the students `[student 2,   student 1,  student 5, student 3,  student 4]` and place it in our matching algorithms startups data, as:



```python
startups_data = {
  "startup_name1": [student 2,   student 1,  student 5, student 3,  student 4],
  ... : ...
}
```




# **Approach 1**
1) Use a key-value mapping of the email and preferences for the students


2) Use a key-value mapping of the email and names for the students (optional)


3) Define the startup based on a key-value mapping for the name, email, and data

In this approach, we use a Python dictionary with the student emails and keys and their preferences for a given startup as the values of the respective keys.

This approach ensures that sorting the data and finding the indices becomes a simpler task.

In [11]:
import numpy as np
from numpy.linalg import norm

# Define the startup profile
startup = {
    'name': 'Dodo',
    'email': 'dodo@example.com',
    'data': np.array([1,0,0,0.8,1,0.5,0.83,0.32,0.1])
}

# Define the student profiles
studentProfile = {
    'alice@example.com': np.array([1,0,0,0,1,1,1,0,0]),
    'bob@example.com': np.array([1,0,0,1,1,1,1,1,0]),
    'charlie@example.com': np.array([1,1,0,1,0,1,0,1,1]),
    'david@example.com': np.array([0,0,1,1,1,1,0,0,0]),
    'eve@example.com': np.array([1,0,0,0,1,1,0,1,0]),
}

# Define the student and startup names
studentNames = {
    'alice@example.com': 'Alice',
    'bob@example.com': 'Bob',
    'charlie@example.com': 'Charlie',
    'david@example.com': 'David',
    'eve@example.com': 'Eve',
}
startupName = startup['name']

# Compute the dot product and cosine similarity
dotProduct = np.dot(list(studentProfile.values()), startup['data'])
cosSim = np.dot(list(studentProfile.values()), startup['data']) / (norm(list(studentProfile.values())) * norm(startup['data']))

# Rank the students by dot product
dotProductRanking = np.argsort(dotProduct)[::-1]
print(f'Dot product ranking for {startupName}:')
for i in dotProductRanking:
    studentEmail = list(studentProfile.keys())[i]
    print(f'{studentNames[studentEmail]} ({studentEmail}): {dotProduct[i]:.2f}')

# Rank the students by cosine similarity
cosSimRanking = np.argsort(cosSim)[::-1]
print(f'\nCosine similarity ranking for {startupName}:')
for i in cosSimRanking:
    studentEmail = list(studentProfile.keys())[i]
    print(f'{studentNames[studentEmail]} ({studentEmail}): {cosSim[i]:.2f}')


Dot product ranking for Dodo:
Bob (bob@example.com): 4.45
Alice (alice@example.com): 3.33
Eve (eve@example.com): 2.82
Charlie (charlie@example.com): 2.72
David (david@example.com): 2.30

Cosine similarity ranking for Dodo:
Bob (bob@example.com): 0.47
Alice (alice@example.com): 0.35
Eve (eve@example.com): 0.30
Charlie (charlie@example.com): 0.29
David (david@example.com): 0.24


# **Approach 2**
Use a key-value mapping of the index and name/email for the students.

After sorting, retrieve the value corresponding to the index based on the above mapping.

In [12]:
import numpy as np
from numpy.linalg import norm

# Define the startup profile
startup = np.array([1,0,0,0.8,1,0.5,0.83,0.32,0.1])

# Define the student profiles
studentProfile = [
    [1,0,0,0,1,1,1,0,0],
    [1,0,0,1,1,1,1,1,0],
    [1,1,0,1,0,1,0,1,1],
    [0,0,1,1,1,1,0,0,0],
    [1,0,0,0,1,1,0,1,0],
]
studProfile = np.array(studentProfile)

# Define the student names
studentNames = {
    0: 'Alice',
    1: 'Bob',
    2: 'Charlie',
    3: 'David',
    4: 'Eve'
}

# Define the startup name
startupName = 'Dodo'

# Compute the dot product and cosine similarity
dotProduct = np.dot(studProfile, startup)
cosSim = np.dot(studProfile, startup) / (norm(studProfile) * norm(startup))

# Rank the students by dot product
dotProductRanking = np.argsort(dotProduct)[::-1]
print(f'Dot product ranking for {startupName}:')
for i in dotProductRanking:
    print(f'{studentNames[i]}: {dotProduct[i]:.2f}')

# Rank the students by cosine similarity
cosSimRanking = np.argsort(cosSim)[::-1]
print(f'\nCosine similarity ranking for {startupName}:')
for i in cosSimRanking:
    print(f'{studentNames[i]}: {cosSim[i]:.2f}')


Dot product ranking for Dodo:
Bob: 4.45
Alice: 3.33
Eve: 2.82
Charlie: 2.72
David: 2.30

Cosine similarity ranking for Dodo:
Bob: 0.47
Alice: 0.35
Eve: 0.30
Charlie: 0.29
David: 0.24


### Building on Approach 2
Now, we try and run the algorithm for n startups and m students based on a sample format. Assuming that the data of students and startups are ready in the format of the samples: startup_data.csv and student_data.csv sheets (links to download the sheets are given below), the following code demonstrates cosine similarity-based matching on sample data for every startup. We write the results to matched_data.csv and it writes the top 10 ranks of students for each startup based on the results of the cosine similarity matching.

To get started, upload the sample data to the Colab notebook by downloading them from here:

[Startup Data](https://drive.google.com/file/d/16b9zjUV4GatRYE4gbGE8SnlaHhPtWqWu/view?usp=sharing)

[Student Data](https://drive.google.com/file/d/12ikqyGIqIGWWjQEwpcmABGSnsV6wWjUQ/view?usp=sharing)

In [13]:
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)

In [18]:
import csv
import os
import numpy as np
from numpy.linalg import norm

# path = '/content/drive/MyDrive/EFL_Collab_Folder/analysisStabMatch/'
path = 'data/'

# Read startup data from CSV
with open(f"{path}startup_data.csv", 'r') as f:
    reader = csv.reader(f)
    startupData = [row for row in reader][1:]
startupNames = [row[0] for row in startupData]
startupProfiles = np.array([row[1:] for row in startupData], dtype=float)

# Read student data from CSV
with open(f'{path}student_data.csv', 'r') as f:
    reader = csv.reader(f)
    studentData = [row for row in reader][1:]
studentNames = [row[0] for row in studentData]
studentProfiles = np.array([row[1:] for row in studentData], dtype=float)

# Compute cosine similarity for each startup
result = {}
for i in range(len(startupNames)):
    # Define the startup profile
    startup = startupProfiles[i]

    # Compute the cosine similarity
    cosSim = np.dot(studentProfiles, startup) / (norm(studentProfiles, axis=1) * norm(startup))

    # Rank the students by cosine similarity
    cosSimRanking = np.argsort(cosSim)[::-1][:10]
    ranking = [(studentNames[j], cosSim[j]) for j in cosSimRanking]
    result[startupNames[i]] = ranking

# Write the results to a CSV file
with open('results/matched_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)

    # Write the header row
    header_row = ['Startup Name']
    for i in range(1, 11):
        header_row.append(f'Rank {i}')
        header_row.append(f'Cosine Similarity {i}')
    writer.writerow(header_row)

    # Write the data rows
    for startup_name in startupNames:
        row = [startup_name]
        for rank, (student_name, cos_sim) in enumerate(result[startup_name], start=1):
            row.append(student_name)
            row.append(cos_sim)
        # Add empty cells for any missing ranks
        for i in range(len(result[startup_name]), 10):
            row.append('')
            row.append('')
        writer.writerow(row)

### Performing the final matching

Based on the results stored in the result dictionary, we need to write code to match each startup with the best student such that the sum of the cosine similarity results of all the pairings of student-startup is maximum. In other words, we find the most stable matching of all pairs.

### The Hungarian Algorithm
One way to find the most stable matching is through The Hungarian Algorithm/The Munkres Assignment Algorithm. The Hungarian algorithm is a combinatorial optimization algorithm used to find the optimal assignment between two sets of items based on a cost or profit matrix. In this case, the algorithm is being used to match startups with students based on their respective profiles and maximize the total cosine similarity of the matched students. The algorithm achieves this by finding the minimum cost assignment of each startup with a student using the similarity matrix, where the cost is the negative of the cosine similarity score.

### Resources to understand the algorithm:
 https://brilliant.org/wiki/hungarian-matching/
  
 https://youtu.be/cQ5MsiGaDY8

### Algorithm explained with an example:
We will assume Startup and Student profiles as follows:

Startup profiles:

Startup|	Marketing	|Tech |
 ---| ---| -- |
Sta1	|8	|6	|3 |
Sta2	| 7	| 9	| 4 |

Student profiles:

Student |	Marketing |	Tech
 ---| ---| -- |
Stu1 |	4 |	8 |	6 |
Stu2|9 |	3 |	5 |

The algorithm will first compute the cosine similarity matrix between the student and startup profiles:

- | Sta1 |	Sta2 |
 ---| ---| -- |
Stu1 |	0.8026 |	0.8924 |
Stu2 |	0.6874 |	0.7348 |

Then it will try to find the optimal one-to-one matching between startups and students to maximize the sum of cosine similarities. The result of running the Hungarian algorithm will be:

- |S1 |	S2 |
 --| ---| ---|
St1	| - |	X |
St2	|X	|- |

This means that startup S1 will be matched with student St2 and startup S2 will be matched with student St1. The total sum of cosine similarities for this matching will be 0.8026 + 0.8924 = 1.6950.

Even though student 1 matched better than student 2 for both startups, the algorithm allocated student 1 with startup 2 instead of startup 1 because the cosine similarity score of student 1 with startup 2 and student 2 with startup 1 respectively is higher than the cosine similarity score of student 1 with startup 1 and student 2 with startup 2 respectively.

### Working code
The following code performs the matching and writes the matched data into the matched_data_hungarian_algorithm.csv file.

In [19]:
import csv
import numpy as np
from numpy.linalg import norm
from scipy.optimize import linear_sum_assignment

# Read startup data from CSV
with open(f'{path}startup_data.csv', 'r') as f:
    reader = csv.reader(f)
    startupData = [row for row in reader][1:]
startupNames = [row[0] for row in startupData]
startupProfiles = np.array([row[1:] for row in startupData], dtype=float)

# Read student data from CSV
with open(f'{path}student_data.csv', 'r') as f:
    reader = csv.reader(f)
    studentData = [row for row in reader][1:]
studentNames = [row[0] for row in studentData]
studentProfiles = np.array([row[1:] for row in studentData], dtype=float)

# Compute cosine similarity for each startup
similarityMatrix = np.dot(studentProfiles, startupProfiles.T) / (norm(studentProfiles, axis=1)[:, np.newaxis] * norm(startupProfiles, axis=1))

# Use Hungarian algorithm to match startups with students
rowIndices, colIndices = linear_sum_assignment(-similarityMatrix)
matchedStudents = {}
for i, j in zip(rowIndices, colIndices):
    startupName = startupNames[j]
    studentName = studentNames[i]
    matchedStudents[startupName] = studentName

# Compute the total sum of cosine similarity for the matched students
totalCosSim = similarityMatrix[rowIndices, colIndices].sum()
print(f'Total sum of cosine similarity for the matched students: {totalCosSim:.2f}')

# Write matched data to CSV
with open('results/matched_data_hungarian_algorithm.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    headers = ['Startup Name', 'Matched Student', 'Cosine Similarity']
    writer.writerow(headers)
    for startupName, studentName in matchedStudents.items():
        cosSim = similarityMatrix[studentNames.index(studentName), startupNames.index(startupName)]
        writer.writerow([startupName, studentName, cosSim])



Total sum of cosine similarity for the matched students: 57.51
